In [1]:
import numpy as np
import pandas as pd
import requests
import gmaps
from config import gkey
import json
import scipy.stats as sts
from matplotlib import pyplot as plt
from pprint import pprint

gmaps.configure(api_key=gkey)

In [2]:
df = pd.read_csv('../WeatherPy/Resources/cities.csv')
df

,Cities,Temperature,Wind Speed,Humidity,Clouds,Latitude,Longitude
0,North Tonawanda,81.00,10.29,44,40,43.04,-78.86
1,Bay Harbor Islands,78.80,8.05,94,1,25.89,-80.13
2,Rhinelander,80.60,5.82,44,20,45.64,-89.41
3,Harrietstown,84.20,3.36,25,1,44.40,-74.16
4,Escuintla,84.20,11.41,83,90,14.30,-90.78
...,...,...,...,...,...,...,...
900,Levittown,79.00,5.82,69,75,40.16,-74.83
901,Fort Polk South,89.60,8.05,46,20,31.05,-93.22
902,Madison,82.00,4.21,51,75,34.73,-86.57
903,Warwick,48.99,5.99,96,21,-28.23,152.02


In [3]:
locations = df[['Latitude', 'Longitude']]
humidity = df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
new_df = df.loc[(df['Temperature'] > 70) & (df['Temperature'] < 80) &
               (df['Wind Speed']< 10) & (df['Clouds']==0)].reset_index().drop(columns=['index'])
locations = new_df[['Latitude', 'Longitude']]
humidity = new_df['Humidity']
new_df.count()
distances_df = new_df[['Latitude',"Longitude"]]
distances_df.reset_index()

new_df

,Cities,Temperature,Wind Speed,Humidity,Clouds,Latitude,Longitude
0,Al Qāmishlī,72.23,7.40,37,0,37.05,41.23
1,Tieling,76.26,5.95,44,0,42.29,123.84
2,Zaghouan,78.01,2.24,68,0,36.40,10.14
3,Beipiao,79.68,3.62,40,0,41.79,120.78
4,Cherskiy,77.65,8.59,27,0,68.75,161.30
5,Surakarta,76.35,6.62,86,0,-7.56,110.83
6,Mahdia,73.40,4.70,83,0,35.50,11.06


In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?query=hotels"
params = {
    "radius" : "5000",
    "type": "lodging",
    "key": gkey
}
hotel_lat = []
hotel_Lng = []
hotel_Name = []
hotel_City = []
hotel_Country = []
for index, row in new_df.iterrows():
    
    coordinates = row[['Latitude','Longitude']]
    latitude = coordinates[0]
    longitude = coordinates[1]
    params['location'] = str(latitude)+','+str(longitude)
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    #print(f"Retrieving Results for Index {index}: {coordinates}.")#
    #try:
    hotel_City.append(row['Cities'])
    hotel_lat.append(response['results'][0]['geometry']['location']['lat'])
    hotel_Lng.append(response['results'][0]['geometry']['location']['lng'])
    hotel_Name.append(response['results'][0]['name'])
    #print(response['results'][0]['plus_code']['compound_code'].split(',')[2])
    #except:
    #print(response['results'][0]['plus_code']['compound_code'].split(',')[1])

In [8]:
#pprint(response['results'][0]['plus_code']['compound_code'].split(',')[2])
pprint(response['results'])

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 35.53205999999999, 'lng': 11.034672},
               'viewport': {'northeast': {'lat': 35.5334629802915,
                                          'lng': 11.0356539302915},
                            'southwest': {'lat': 35.5307650197085,
                                          'lng': 11.03295596970849}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': 'cec44863c5df733a3fb491c96f0d4ba3c80b9e7a',
  'name': 'Mahdia Beach',
  'photos': [{'height': 3096,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/114609666321377640584">Nidhal '
                                    'Bouzidi</a>'],
              'photo_reference': 'CmRaAAAAoF6_0rKz5G0Na7ZGm8RHP1t2PTaasS3r_sqoQcqBKIEQZS2mUjpso3YzOQgZyRWGQlQznhnIDtPCJ9LnoHkiNdiBRfjZbOGR1Hw-7oVEkU7ITGRHn99nbSMLvSA4r-YuEhCqhc-q1XV6dY6bC3Ow7o4-GhREnBj1jkYt6T6SoAQ_RA0MaJfnpg',

In [18]:
print(len(hotel_lat))
print(len(hotel_Lng))
print(len(hotel_Name))
print(len(hotel_City))

7
7
7
7


In [19]:
hotel_df = pd.DataFrame({'Hotel Name':hotel_Name,
                         'Hotel Long':hotel_Lng,
                         'Hotel Lat':hotel_lat,
                         'Hotel City':hotel_City
                        })

hotel_df

,Hotel Name,Hotel Long,Hotel Lat,Hotel City
0,Kasr-ı Serçehan Hotel,41.216478,37.068070,Al Qāmishlī
1,Tieling 99 Yuejia Business Guest House,123.845414,42.297050,Tieling
2,Dar Zaghouan,10.123882,36.416293,Zaghouan
3,Beipiao Hotel,120.767309,41.803353,Beipiao
4,Gostinitsa,161.327610,68.756827,Cherskiy
5,The Sunan Hotel,110.794316,-7.557335,Surakarta
6,Mahdia Beach,11.034672,35.532060,Mahdia


In [22]:
locations = df[['Latitude', 'Longitude']]
hotel_locations = hotel_df[['Hotel Lat','Hotel Long']]
humidity = df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100,
                                 point_radius=1)
markers = gmaps.marker_layer(hotel_locations)

# Display figure
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))